<a href="https://colab.research.google.com/github/nuvard/Where-to-eat/blob/master/data_preparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/nuvard/Where-to-eat.git
  

Cloning into 'Where-to-eat'...
remote: Enumerating objects: 18, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 18 (delta 3), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (18/18), done.


In [0]:
import numpy as np

In [0]:
import pandas as pd
path = '/content/Where-to-eat/data/'
rests = pd.read_csv(path+'yandex_rests.csv', index_col=0)

In [243]:
rests.head()

,address,features,lat,link,lng,name,rating_yandex,review_count,reviews
0,пр. Луначарского 60,Посмотреть всё меню|До закрытия 7 мин|Нет инфо...,"60.041867,",NaN,30.340230,Кафе-бар Белые ночи,4.2,28,Забегаловка . Но вечером может быть весело|Хор...
1,наб. реки Фонтанки 59,Посмотреть всё меню|Закрыто до 14:00|+7 (812) ...,"59.927757,",NaN,30.332271,White Night Music Joint,4.1,105,"Хорошая кухня, адекватные цены, джаз. Но интер..."
2,наб. реки Фонтанки 59,Посмотреть всё меню|Закрыто до 9:00|+7 (981) 8...,"59.927757,",NaN,30.332271,Kiosk59,4.4,69,"Ещё одна кофейня, которую нужно посетить любит..."
3,наб. реки Пряжки 30,Посмотреть всё меню|Закрыто до 7:00|+7 (812) 6...,"59.922792,",NaN,30.282002,Французский дворик,2.7,8,Бар-кафе при гостинице КРАСНЫЕ ЗВЕЗДЫ. Вежливы...
4,наб. реки Пряжки 30,До 50% на кухню|Посмотреть всё меню|Открыто до...,"59.922792,",NaN,30.282002,Beer House Staff,4.3,135,"Мой отзыв от 12.04.19\r\r\n""Обслуживание так с..."


In [244]:
rests.shape

(1983, 9)

In [245]:
rests.columns

Index(['address', 'features', 'lat', 'link', 'lng', 'name', 'rating_yandex',
       'review_count', 'reviews'],
      dtype='object')

In [0]:
rests['lat'] = rests['lat'].apply(lambda x: x.replace(',',''))


In [0]:
rests['lat'] = rests['lat'].astype('float')
rests['lng'] = rests['lng'].astype('float')

In [248]:
np.mean(rests.loc[rests.review_count!='Ещё', 'review_count'].astype(float))

183.29299363057325

In [0]:
rests.review_count = rests.review_count.replace('Ещё', 183)

In [0]:
rests.review_count = rests.review_count.astype('float')

In [251]:
rests.dtypes

address           object
features          object
lat              float64
link              object
lng              float64
name              object
rating_yandex    float64
review_count     float64
reviews           object
dtype: object

In [252]:
rests.describe()

,lat,lng,rating_yandex,review_count
count,1983.000000,1983.000000,1983.000000,1983.000000
mean,59.935549,30.322041,4.175693,183.278366
std,0.044011,0.082520,0.533513,228.999027
min,59.566154,29.001130,1.200000,3.000000
25%,59.926988,30.306980,4.100000,56.000000
50%,59.935290,30.326926,4.300000,125.000000
75%,59.944508,30.351558,4.400000,198.000000
max,60.231007,30.513596,5.000000,2415.000000


In [0]:
from sklearn.preprocessing import StandardScaler, Normalizer, MinMaxScaler

In [0]:
rests[['rating_yandex_norm','review_count_norm']] = rests[['rating_yandex','review_count']]

In [0]:
scaler = MinMaxScaler()
rating_scaler = scaler.fit(rests[['rating_yandex','review_count']])
rests[['rating_yandex_norm','review_count_norm']] = rating_scaler.transform(rests[['rating_yandex','review_count']])


In [0]:
rests['popularity']=rests.apply(lambda x: x['rating_yandex_norm']*x['review_count_norm'], axis=1)
rating_scaler = scaler.fit(np.array(rests['popularity']).reshape(-1, 1))
rests['popularity'] = rating_scaler.transform(np.array(rests['popularity']).reshape(-1, 1))


In [257]:
rests.head()

,address,features,lat,link,lng,name,rating_yandex,review_count,reviews,rating_yandex_norm,review_count_norm,popularity
0,пр. Луначарского 60,Посмотреть всё меню|До закрытия 7 мин|Нет инфо...,60.041867,NaN,30.340230,Кафе-бар Белые ночи,4.2,28.0,Забегаловка . Но вечером может быть весело|Хор...,0.789474,0.010365,0.010443
1,наб. реки Фонтанки 59,Посмотреть всё меню|Закрыто до 14:00|+7 (812) ...,59.927757,NaN,30.332271,White Night Music Joint,4.1,105.0,"Хорошая кухня, адекватные цены, джаз. Но интер...",0.763158,0.042289,0.041186
2,наб. реки Фонтанки 59,Посмотреть всё меню|Закрыто до 9:00|+7 (981) 8...,59.927757,NaN,30.332271,Kiosk59,4.4,69.0,"Ещё одна кофейня, которую нужно посетить любит...",0.842105,0.027363,0.029407
3,наб. реки Пряжки 30,Посмотреть всё меню|Закрыто до 7:00|+7 (812) 6...,59.922792,NaN,30.282002,Французский дворик,2.7,8.0,Бар-кафе при гостинице КРАСНЫЕ ЗВЕЗДЫ. Вежливы...,0.394737,0.002073,0.001044
4,наб. реки Пряжки 30,До 50% на кухню|Посмотреть всё меню|Открыто до...,59.922792,NaN,30.282002,Beer House Staff,4.3,135.0,"Мой отзыв от 12.04.19\r\r\n""Обслуживание так с...",0.815789,0.054726,0.056976


In [0]:
rests.drop_duplicates(subset = list(rests.columns[[0,5]]), inplace=True)

In [259]:
rests.shape

(1067, 12)

In [260]:
list(rests.columns[[0,5]])

['address', 'name']

In [0]:
rests.features = rests.features.fillna('')

In [0]:
rests['features']=rests['features'].apply(lambda x: x.split('|'))

In [263]:

rests.dtypes

address                object
features               object
lat                   float64
link                   object
lng                   float64
name                   object
rating_yandex         float64
review_count          float64
reviews                object
rating_yandex_norm    float64
review_count_norm     float64
popularity            float64
dtype: object

In [0]:
bad_features = ['ул.', '+7', 'пр.', 'ш.', 'Показать телефон', '.ru', ' м', 'Посмотреть всё меню', '?', \
               'Закрыто до','До открытия ', 'Посмотреть всё меню', 'просп', 'Соцсети', 'Подробнее', 'com','Нет информации', 'Все филиалы сети',\
               'наб', 'Добавить организацию', 'Показать парковки','смотреть', '.fi','.pro','.info', 'смотреть', 'пер.', 'км', 'фото'\
               'линия', '.su', 'Открыто до','₽', 'Где', 'Еще', 'подбор', 'площадь', 'Показать входы','!','Кухня:', 'Особенности заведения:' ]
def clean(features):
  good_features = []
  counter = 0;
  for feature in features:
      if(feature.find('Кухня:')!=-1):
          good_features.extend(feature.split(':')[1].replace(' ','кухня_').split(','))
          #features.pop(feature)
      if(feature.find('Особенности заведения:')!=-1):
          good_features.extend(feature.split(':')[1].split(', '))
      if feature.find('%')!=-1:
          feature = 'discount_'+str(abs(int(feature.split('%')[0].split(' ')[-1])))
      if feature.lower().find('круглосуточно')!=-1:
            feature = '24_hour'
             
      for i in bad_features:
          
          if feature.find(i)!=-1:
              counter+=1
              continue 
          
      if counter==0:
          good_features.append(feature)
  print(set(good_features[0:100]))
  return list(set(good_features))

def find_kitchen(features):
    for feature in features:
        if(feature.find('Кухня:')!=-1):
            features.append(feature.split(':')[1].replace(' ','кухня_').split(','))
            features.pop(feature)
    return features
  
  
def find_features(features):
    for feature in features:
        if(feature.find('Особенности заведения:')!=-1):
            features.append(feature.split(':')[1].split(', '))
            features.pop(feature)
    return features


In [0]:
rests['kitchen'] = ''

In [268]:
rests.tail()

,address,features,lat,link,lng,name,rating_yandex,review_count,reviews,rating_yandex_norm,review_count_norm,popularity,kitchen
1977,пр. Московский 139/1,"[Посмотреть всё меню, Открыто до 2:00, +7 (812...",59.879864,https://yandex.ru/maps/org/gent/112945565451/,30.318527,Гент,4.4,170.0,"Так, начнём! Моя продюсерская судьба кидала ме...",0.842105,0.069237,0.074408,
1978,пр. Каменноостровский 22,"[Посмотреть всё меню, Открыто до 23:00, +7 (81...",59.960791,https://yandex.ru/maps/org/coffee_room/1155017...,30.315626,Coffee Room,4.2,132.0,Кафе посетили по старой памяти. Ранее всегда ц...,0.789474,0.053483,0.053885,
1979,ул. Савушкина 21 лит. А,"[Посмотреть всё меню, До открытия 9 мин, +7 (8...",59.986079,https://yandex.ru/maps/org/yunost/158639523050/,30.285828,Юность,4.7,629.0,"Понтов много, ценник высокий, в субботу днем п...",0.921053,0.259536,0.305068,
1981,пер. Перекупной 3,"[Открыто до 23:00, +7 (911) 905-10-04\nПоказат...",59.927617,https://yandex.ru/maps/org/bombay/86669928054/,30.376774,BombaY,4.3,183.0,"На любителя.Зловонные приправы,не опытные пова...",0.815789,0.074627,0.077694,
1982,В.О. пр. Большой 19,"[Посмотреть всё меню, До открытия 8 мин, +7 (8...",59.940078,https://yandex.ru/maps/org/tan_zhen/1042818239/,30.283987,Тан Жен,4.4,431.0,"Здоровский ресторан, но понимаете, что размер ...",0.842105,0.177446,0.190699,


In [271]:
#rests.kitchen = rests.features.apply(find_kitchen)
#rests.features = rests.features.apply(find_features)
rests.features = rests.features.apply(clean)

{'кухня_грузинская', 'кухня_кавказская', 'кухня_европейская'}
{'кухня_итальянская', 'кухня_русская', 'кухня_европейская'}
set()
{'кухня_итальянская', 'закрытие под банкет', 'барная стойка', ' меню на английском', 'кухня_европейская', 'кухня_грузинская'}
{'кухня_итальянская', 'кухня_русская', 'discount_50', 'кухня_ирландская'}
{'кухня_русская', 'кухня_европейская'}
{'кухня_тайская', 'кухня_китайская', 'кухня_корейская'}
set()
{'кухня_английская', 'настольные игры', 'кухня_авторская', 'барная стойка', ' меню на английском', 'кухня_мясная'}
{'кухня_итальянская', ' меню на английском', 'барная стойка', 'кухня_европейская'}
set()
{'кухня_европейская'}
{'кухня_грузинская', 'кухня_кавказская', 'кухня_европейская'}
{'кухня_русская', 'кухня_европейская'}
{'кухня_русская', 'кухня_европейская', 'discount_50', 'кухня_мясная'}
{'кухня_европейская'}
{'кухня_итальянская', 'кухня_европейская', 'кухня_фьюжн'}
{'кухня_русская', ' своя пекарня', 'закрытие под банкет'}
{'кухня_смешанная', ' закрытие под б

In [0]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
temp = mlb.fit_transform(rests['features'])
temp = pd.concat([rests,pd.DataFrame(temp, columns = mlb.classes_).fillna(0)], axis=1)

In [274]:
temp.head()

,address,features,lat,link,lng,name,rating_yandex,review_count,reviews,rating_yandex_norm,review_count_norm,popularity,kitchen,DJ,барная стойка,бесплатная парковка,винная карта,живая музыка,закрытие под банкет,круглосуточная кухня,меню на английском,местоположение у воды,настольные игры,своя пекарня,устричный бар,24_hour,DJ,Restopub Finnegan's приглашает на бизнес ланч,discount_10,discount_15,discount_20,discount_50,discount_7,Завтраки в ресторане МА,Счастливые часы,барная стойка,бесплатная парковка,бранчи,винная карта,детская анимация,...,кухня_домашняя,кухня_еврейская,кухня_европейская,кухня_индийская,кухня_индонезийская,кухня_ирландская,кухня_испанская,кухня_итальянская,кухня_кавказская,кухня_китайская,кухня_континентальная,кухня_корейская,кухня_латиноамериканская,кухня_международная,кухня_мексиканская,кухня_морская,кухня_мясная,кухня_национальная,кухня_немецкая,кухня_неткухня_специализации,кухня_паназиатская,кухня_русская,кухня_рыбная,кухня_сербская,кухня_скандинавская,кухня_смешанная,кухня_средиземноморская,кухня_тайская,кухня_узбекская,кухня_украинская,кухня_французская,кухня_фьюжн,кухня_чешская,кухня_шашлык,кухня_японская,меню на английском,местоположение у воды,настольные игры,своя пекарня,танцпол
0,пр. Луначарского 60,"[кухня_грузинская, кухня_кавказская, кухня_евр...",60.041867,NaN,30.340230,Кафе-бар Белые ночи,4.2,28.0,Забегаловка . Но вечером может быть весело|Хор...,0.789474,0.010365,0.010443,,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,наб. реки Фонтанки 59,"[кухня_итальянская, кухня_русская, кухня_европ...",59.927757,NaN,30.332271,White Night Music Joint,4.1,105.0,"Хорошая кухня, адекватные цены, джаз. Но интер...",0.763158,0.042289,0.041186,,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,наб. реки Фонтанки 59,[],59.927757,NaN,30.332271,Kiosk59,4.4,69.0,"Ещё одна кофейня, которую нужно посетить любит...",0.842105,0.027363,0.029407,,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,наб. реки Пряжки 30,"[кухня_итальянская, закрытие под банкет, барна...",59.922792,NaN,30.282002,Французский дворик,2.7,8.0,Бар-кафе при гостинице КРАСНЫЕ ЗВЕЗДЫ. Вежливы...,0.394737,0.002073,0.001044,,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,наб. реки Пряжки 30,"[кухня_итальянская, кухня_русская, discount_50...",59.922792,NaN,30.282002,Beer House Staff,4.3,135.0,"Мой отзыв от 12.04.19\r\r\n""Обслуживание так с...",0.815789,0.054726,0.056976,,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [275]:
rests = temp
rests.pop('features')

0       [кухня_грузинская, кухня_кавказская, кухня_евр...
1       [кухня_итальянская, кухня_русская, кухня_европ...
2                                                      []
3       [кухня_итальянская, закрытие под банкет, барна...
4       [кухня_итальянская, кухня_русская, discount_50...
5                      [кухня_русская, кухня_европейская]
6       [кухня_тайская, кухня_китайская, кухня_корейская]
7                                                      []
8       [кухня_английская, настольные игры, кухня_авто...
9       [кухня_итальянская,  меню на английском, барна...
10                                                     []
11                                    [кухня_европейская]
12      [кухня_грузинская, кухня_кавказская, кухня_евр...
13                     [кухня_русская, кухня_европейская]
14      [кухня_русская, кухня_европейская, discount_50...
15                                    [кухня_европейская]
16      [кухня_итальянская, кухня_европейская, кухня_ф...
17      [кухня

In [0]:
rests.to_csv('yandex_rests_preprocessed.csv', index = False)

In [241]:
rests['kitchen'].fillna([]).replace(0, [])

TypeError: ignored

In [238]:
kitchen_mlb = MultiLabelBinarizer()
temp = kitchen_mlb.fit_transform(rests.kitchen)
kitchen_mlb.classes_
temp = pd.concat([rests,pd.DataFrame(temp, columns = kitchen_mlb.classes_)], axis=1)

TypeError: ignored

In [172]:
rests[kitchen_mlb.classes_] = pd.DataFrame(temp, columns = kitchen_mlb.classes_)

,кухня_авторская,кухня_азербайджанская,кухня_азиатская,кухня_американская,кухня_английская,кухня_армянская,кухня_белорусская,кухня_веганская,кухня_вегетарианская,кухня_восточная,кухня_греческая,кухня_грузинская,кухня_домашняя,кухня_еврейская,кухня_европейская,кухня_индийская,кухня_индонезийская,кухня_ирландская,кухня_испанская,кухня_итальянская,кухня_кавказская,кухня_китайская,кухня_континентальная,кухня_корейская,кухня_латиноамериканская,кухня_международная,кухня_мексиканская,кухня_морская,кухня_мясная,кухня_национальная,кухня_немецкая,кухня_неткухня_специализации,кухня_паназиатская,кухня_русская,кухня_рыбная,кухня_сербская,кухня_скандинавская,кухня_смешанная,кухня_средиземноморская,кухня_тайская,кухня_узбекская,кухня_украинская,кухня_французская,кухня_фьюжн,кухня_чешская,кухня_шашлык,кухня_японская
0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
